If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

pip install -U transformers accelerate bitsandbytes

In [1]:
import os
os.environ['HF_HOME'] = 'D:/cache/'

In [3]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-24 22:12:49--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-07-24 22:12:50 (2.66 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer
from huggingface_hub import login
import os

In [7]:
login(token=os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/bayus/.cache/huggingface/token
Login successful


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [14]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    return index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3,
    )

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
QUESTION: {question}

CONTEXT:
{context}

ANSWER:
""".strip()

    context = ""

    for doc in search_results:
        context = f"{context}{doc['question']}\n{doc['text']}\n\n"

    return prompt_template.format(question=query, context=context).strip()

def llm(prompt):
    response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()

In [16]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [17]:
rag("I just discovered the course. Can I still join it?")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/mnt/d/repos/llm-zoomcamp/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/d/repos/llm-zoomcamp/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p

'Yes, you can still join the course.'

In [27]:
!df -h

Filesystem                                Size  Used Avail Use% Mounted on
none                                      7.8G  4.0K  7.8G   1% /mnt/wsl
none                                      477G  468G  8.6G  99% /usr/lib/wsl/drivers
/dev/sdc                                 1007G  100G  857G  11% /
none                                      7.8G   88K  7.8G   1% /mnt/wslg
none                                      7.8G     0  7.8G   0% /usr/lib/wsl/lib
rootfs                                    7.8G  2.1M  7.8G   1% /init
none                                      7.8G  776K  7.8G   1% /run
none                                      7.8G     0  7.8G   0% /run/lock
none                                      7.8G     0  7.8G   0% /run/shm
tmpfs                                     4.0M     0  4.0M   0% /sys/fs/cgroup
none                                      7.8G   76K  7.8G   1% /mnt/wslg/versions.txt
none                                      7.8G   76K  7.8G   1% /mnt/wslg/doc
C:\             